# Healthcare Data Analysis and Visualization


**Project Goal**: Perform comprehensive Pandas analysis including pivot tables, categorical analysis, and distribution tables to uncover relationships in the healthcare dataset.

**Dataset**: NSMES1988updated.csv (from Session 2 - with corrected scaling)

---

## Session 3: Data Analysis with Pandas Tasks

**Dataset**: NSMES1988updated.csv

### Part 1: Working with Pandas
**Tasks:**
1. Import relevant Python libraries necessary for Python and Pandas analysis.
2. Import the CSV file NSMES1988updated.csv file and create a new dataframe for working with Pandas.
3. Identify different types of data and report it.
4. Identify Categorical types in the data.
5. Perform a detailed Data pivoting on the dataframe and report it.
6. Include the following categorical data in your analysis:
   - Health
   - Region
7. Prepare a detailed report on your analysis and observations.

### Part 2: Analyze and Cleanse Data
**Tasks:**
1. Import relevant Python libraries necessary for Python and Pandas analysis.
2. Import the CSV file NSMES1988updated.csv file and create a new dataframe for working with Pandas.
3. Perform analysis based on the following criteria: Different types of visits, Gender, Marital Status, School, Income, Employment Status, Insurance, Medical Aid
4. Explore and analyze the dataset to gain insights into how different factors relate to each other. Create distribution tables:
   - Age and Gender Distribution
   - Health Status by Gender
   - Income Distribution by Gender
   - Regional Income Distribution
   - Age-wise Income Analysis
5. Report your findings.

---

---
## Part 1: Working with Pandas

### Tasks 1-2: Library Import and Data Loading

In [9]:
# import libraries and load data

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# load the transformed data from Session 2
df = pd.read_csv('outputs/NSMES1988updated.csv')

# drop Unnamed: 0 if exists
if 'Unnamed: 0' in df.columns:
    df = df.drop('Unnamed: 0', axis=1)

# re-apply categorical optimizations (CSV doesn't preserve types)
categorical_cols = ['health', 'gender', 'married', 'region', 
                    'employed', 'insurance', 'medicaid', 'adl']
for col in categorical_cols:
    if col in df.columns:
        df[col] = df[col].astype('category')

print(f"Data loaded: {df.shape[0]} rows, {df.shape[1]} columns")
df.head()

Data loaded: 4406 rows, 18 columns


,visits,nvisits,ovisits,novisits,emergency,hospital,health,chronic,adl,region,age,gender,married,school,income,employed,insurance,medicaid
0,5,0,0,0,0,1,average,2,normal,other,69.0,male,yes,6,28810.0,yes,yes,no
1,1,0,2,0,2,0,average,2,normal,other,74.0,female,yes,10,27478.0,no,yes,no
2,13,0,0,0,3,3,poor,4,limited,other,66.0,female,no,10,6532.0,no,no,yes
3,16,0,5,0,1,1,poor,2,limited,other,76.0,male,yes,3,6588.0,no,yes,no
4,3,0,0,0,0,0,average,2,limited,other,79.0,female,yes,6,6588.0,no,yes,no


In [16]:
# Create categorical groupings for analysis
df['age_group'] = pd.cut(df['age'], 
                          bins=[65, 70, 75, 80, 85, 110],
                          labels=['66-70', '71-75', '76-80', '81-85', '86+'])

df['income_group'] = pd.cut(df['income'],
                             bins=[-20000, 10000, 20000, 30000, 50000, 600000],
                             labels=['<$10k', '$10k-$20k', '$20k-$30k', '$30k-$50k', '$50k+'])


### Task 3: Identify Different Types of Data

In [10]:
# separate by type
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=['category', 'object']).columns.tolist()

print("\n" + "="*50)
print("DATA TYPE SUMMARY:")
print("-"*50)
print(f"\nNumerical Columns ({len(numeric_cols)}):\n")
for col in numeric_cols:
    print(f"  • {col}: {df[col].dtype}")

print(f"\nCategorical Columns ({len(categorical_cols)}):\n")
for col in categorical_cols:
    print(f"  • {col}: {df[col].dtype}")

print("\n" + "="*50)


DATA TYPE SUMMARY:
--------------------------------------------------

Numerical Columns (10):

  • visits: int64
  • nvisits: int64
  • ovisits: int64
  • novisits: int64
  • emergency: int64
  • hospital: int64
  • chronic: int64
  • age: float64
  • school: int64
  • income: float64

Categorical Columns (8):

  • health: category
  • adl: category
  • region: category
  • gender: category
  • married: category
  • employed: category
  • insurance: category
  • medicaid: category



### Task 4: Identify Categorical Types in Detail

In [11]:
print("="*50)
print("CATEGORICAL VARIABLES DETAILED ANALYSIS:")
print("-"*50)

categorical_info = []

for col in categorical_cols:
    unique_count = df[col].nunique()
    categories = df[col].unique()
    value_counts = df[col].value_counts()
    
    print(f"\n{col.upper()}:")
    print(f"  Unique values: {unique_count}")
    print(f"  Categories: {list(categories)}")
    print(f"  Distribution:")
    for category, count in value_counts.items():
        pct = (count / len(df)) * 100
        print(f"    - {category}: {count} ({pct:.1f}%)")
    
    categorical_info.append({
        'Column': col,
        'Unique Values': unique_count,
        'Most Common': value_counts.index[0],
        'Most Common Count': value_counts.iloc[0],
        'Most Common %': f"{(value_counts.iloc[0] / len(df) * 100):.1f}%"
    })

# summary table
print("\n" + "="*50)
print("CATEGORICAL SUMMARY TABLE:\n")
cat_summary = pd.DataFrame(categorical_info)
print(cat_summary.to_string(index=False))

CATEGORICAL VARIABLES DETAILED ANALYSIS:
--------------------------------------------------

HEALTH:
  Unique values: 3
  Categories: ['average', 'poor', 'excellent']
  Distribution:
    - average: 3509 (79.6%)
    - poor: 554 (12.6%)
    - excellent: 343 (7.8%)

ADL:
  Unique values: 2
  Categories: ['normal', 'limited']
  Distribution:
    - normal: 3507 (79.6%)
    - limited: 899 (20.4%)

REGION:
  Unique values: 4
  Categories: ['other', 'midwest', 'northeast', 'west']
  Distribution:
    - other: 1614 (36.6%)
    - midwest: 1157 (26.3%)
    - northeast: 837 (19.0%)
    - west: 798 (18.1%)

GENDER:
  Unique values: 2
  Categories: ['male', 'female']
  Distribution:
    - female: 2628 (59.6%)
    - male: 1778 (40.4%)

MARRIED:
  Unique values: 2
  Categories: ['yes', 'no']
  Distribution:
    - yes: 2406 (54.6%)
    - no: 2000 (45.4%)

EMPLOYED:
  Unique values: 2
  Categories: ['yes', 'no']
  Distribution:
    - no: 3951 (89.7%)
    - yes: 455 (10.3%)

INSURANCE:
  Unique values: 2

### Tasks 3-4: Data Types Observations

**Numerical Variables** (10 columns):
- These represent measurable quantities (age, income, visits, etc.)
- Can perform statistical operations (mean, median, sum)
- Range from counts (visits, chronic conditions) to continuous measures (age, income)

**Categorical Variables** (8 columns):
- Represent groups or classifications
- Include demographics (gender, region, marital status)
- Include health-related categories (health status, ADL limitations)
- Include economic indicators (employment, insurance, medicaid)

**Key Categorical Distributions:**
- **Gender**: 59.6% female, 40.4% male
- **Health Status**: Most common is "average" (79.6%)
- **Region**: Distributed across 4 regions, "other" most common (36.6%)
- **Insurance**: 77.6% have private insurance
- **Medicaid**: 90.9% on medicaid

These categorical variables will be used as grouping dimensions in the pivot table analysis.

---
### Tasks 5-6: Data Pivoting Analysis

Perform detailed pivot table analysis using Health and Region as categorical dimensions.

In [12]:
# Tasks 5-6: Pivot table analysis with Health category

print("="*50)
print("PIVOT TABLE ANALYSIS: HEALTH STATUS")
print("-"*50)

# 1. health vs healthcare utilization
print("\n1. AVERAGE HEALTHCARE UTILIZATION BY HEALTH STATUS:")
print("-"*50)

health_pivot = df.groupby('health').agg({
    'visits': 'mean',
    'nvisits': 'mean',
    'ovisits': 'mean',
    'novisits': 'mean',
    'emergency': 'mean',
    'hospital': 'mean',
    'chronic': 'mean'
}).round(2)

print(health_pivot)

# 2. health vs demographics
print("\n\n2. DEMOGRAPHICS BY HEALTH STATUS:")
print("-"*50)

health_demo = df.groupby('health').agg({
    'age': 'mean',
    'income': 'mean',
    'school': 'mean'
}).round(2)

health_demo['income'] = health_demo['income'].apply(lambda x: f"${x:,.2f}")

print(health_demo)

# 3. health distribution with counts
print("\n\n3. HEALTH STATUS DISTRIBUTION:")
print("-"*50)

health_counts = df['health'].value_counts()
health_pct = (df['health'].value_counts(normalize=True) * 100).round(1)

health_dist = pd.DataFrame({
    'Count': health_counts,
    'Percentage': health_pct.apply(lambda x: f"{x}%")
})

print(health_dist)

# 4. Cross-tab: Health vs Gender
print("\n\n4. HEALTH STATUS BY GENDER:")
print("-"*50)

health_gender = pd.crosstab(
    df['health'], 
    df['gender'], 
    margins=True, 
    margins_name='Total'
)

print(health_gender)

PIVOT TABLE ANALYSIS: HEALTH STATUS
--------------------------------------------------

1. AVERAGE HEALTHCARE UTILIZATION BY HEALTH STATUS:
--------------------------------------------------
           visits  nvisits  ovisits  novisits  emergency  hospital  chronic
health                                                                     
average      5.51     1.65     0.69      0.55       0.22      0.25     1.43
excellent    3.43     1.59     0.34      0.16       0.10      0.10     0.75
poor         8.90     1.42     1.38      0.69       0.62      0.69     2.74


2. DEMOGRAPHICS BY HEALTH STATUS:
--------------------------------------------------
             age      income  school
health                              
average    73.85  $25,519.90   10.43
excellent  73.10  $32,774.11   11.33
poor       75.70  $19,051.62    8.78


3. HEALTH STATUS DISTRIBUTION:
--------------------------------------------------
           Count Percentage
health                     
average     3509 

In [13]:
# Tasks 5-6: Pivot table analysis with Region category

print("\n" + "="*50)
print("PIVOT TABLE ANALYSIS: REGION")
print("-"*50)

# 1. region vs healthcare utilization
print("\n1. AVERAGE HEALTHCARE UTILIZATION BY REGION:")
print("-"*50)

region_pivot = df.groupby('region').agg({
    'visits': 'mean',
    'nvisits': 'mean',
    'ovisits': 'mean',
    'novisits': 'mean',
    'emergency': 'mean',
    'hospital': 'mean',
    'chronic': 'mean'
}).round(2)

print(region_pivot)

# 2. region vs demographics
print("\n\n2. DEMOGRAPHICS BY REGION:")
print("-"*50)

region_demo = df.groupby('region').agg({
    'age': 'mean',
    'income': 'mean',
    'school': 'mean'
}).round(2)

region_demo['income'] = region_demo['income'].apply(lambda x: f"${x:,.2f}")

print(region_demo)

# 3. region distribution
print("\n\n3. REGIONAL DISTRIBUTION:")
print("-"*50)

region_counts = df['region'].value_counts()
region_pct = (df['region'].value_counts(normalize=True) * 100).round(1)

region_dist = pd.DataFrame({
    'Count': region_counts,
    'Percentage': region_pct.apply(lambda x: f"{x}%")
})

print(region_dist)

# 4. Cross-tab: Region vs Insurance Coverage
print("\n\n4. INSURANCE COVERAGE BY REGION:")
print("-"*50)

region_insurance = pd.crosstab(
    df['region'], 
    df['insurance'], 
    margins=True, 
    margins_name='Total'
)

print(region_insurance)

# 5. region vs medicaid
print("\n\n5. MEDICAID COVERAGE BY REGION:")
print("-"*50)

region_medicaid = pd.crosstab(
    df['region'], 
    df['medicaid'], 
    margins=True, 
    margins_name='Total'
)

print(region_medicaid)


PIVOT TABLE ANALYSIS: REGION
--------------------------------------------------

1. AVERAGE HEALTHCARE UTILIZATION BY REGION:
--------------------------------------------------
           visits  nvisits  ovisits  novisits  emergency  hospital  chronic
region                                                                     
midwest      5.41     1.95     0.71      0.64       0.24      0.30     1.46
northeast    6.09     1.66     0.80      0.50       0.25      0.27     1.49
other        5.58     1.00     0.76      0.58       0.28      0.30     1.64
west         6.37     2.34     0.74      0.34       0.28      0.30     1.52


2. DEMOGRAPHICS BY REGION:
--------------------------------------------------
             age      income  school
region                              
midwest    74.08  $25,136.34   10.41
northeast  73.93  $26,797.09   10.34
other      73.99  $21,662.84    9.59
west       74.12  $31,165.05   11.48


3. REGIONAL DISTRIBUTION:
------------------------------------

In [14]:
# combined pivot: Health and Region interaction

print("\n" + "="*50)
print("COMBINED PIVOT ANALYSIS: HEALTH × REGION")
print("-"*50)

# 1. Multi-index pivot: Average visits by Health and Region
print("\n1. AVERAGE PHYSICIAN VISITS BY HEALTH STATUS AND REGION:")
print("-"*50)

combined_visits = df.pivot_table(
    values='visits',
    index='health',
    columns='region',
    aggfunc='mean',
    margins=True,
    margins_name='Overall'
).round(2)

print(combined_visits)

# 2. Income by Health and Region
print("\n\n2. AVERAGE INCOME BY HEALTH STATUS AND REGION:")
print("-"*50)

combined_income = df.pivot_table(
    values='income',
    index='health',
    columns='region',
    aggfunc='mean',
    margins=True,
    margins_name='Overall'
).round(2)

print(combined_income)

# 3. Count distribution
print("\n\n3. POPULATION COUNT BY HEALTH STATUS AND REGION:")
print("-"*50)

combined_count = pd.crosstab(
    df['health'],
    df['region'],
    margins=True,
    margins_name='Total'
)

print(combined_count)

# 4. Chronic conditions by Health and Region
print("\n\n4. AVERAGE CHRONIC CONDITIONS BY HEALTH STATUS AND REGION:")
print("-"*50)

combined_chronic = df.pivot_table(
    values='chronic',
    index='health',
    columns='region',
    aggfunc='mean',
    margins=True,
    margins_name='Overall'
).round(2)

print(combined_chronic)


COMBINED PIVOT ANALYSIS: HEALTH × REGION
--------------------------------------------------

1. AVERAGE PHYSICIAN VISITS BY HEALTH STATUS AND REGION:
--------------------------------------------------
region     midwest  northeast  other  west  Overall
health                                             
average       5.28       5.69   5.09  6.50     5.51
excellent     3.44       4.02   3.14  3.37     3.43
poor          8.12      10.67   8.75  8.59     8.90
Overall       5.41       6.09   5.58  6.37     5.77


2. AVERAGE INCOME BY HEALTH STATUS AND REGION:
--------------------------------------------------
region      midwest  northeast     other      west   Overall
health                                                      
average    24984.23   27015.35  22010.96  31663.77  25519.90
excellent  31053.88   33400.12  30024.58  37255.87  32774.11
poor       21618.12   20659.30  16851.79  21118.85  19051.62
Overall    25136.34   26797.09  21662.84  31165.05  25271.32


3. POPULATION COUN

## Tasks 5-6: Pivot Table Analysis Report

#### Health Status Analysis

**Key Findings:**

1. **Healthcare Utilization by Health Status:**
   - People with poor health also have significantly more visits.
   - Those with average health were the next most common with those with excellent had the least.
   - Emergency visit and hospital stays were almost the same, with those with average health staying at the hospital 0.03 more, and those with poor health staying at the hospital 0.07 more. 

2. **Demographics by Health Status:**
   - Average age: 73 for those with average and excellent health, 75 for those with poor health.
   - Average income: \\$19,051 for those with poor health, \\$25,519 for those with average health and \\$32,774 for those with excellent health.
   - Education: Poor health had an average of 8.78 years, average had 10.45, and excellent had 11.33.

3. **Health Distribution:**
   - Most people report average health (79.6%)
   - 12.6% report poor health, 7.8% report excellent health

4. **Health by Gender:**
   - There are more women (2628) than men (1778) and women report more than men in every category.
   - Women report average 677 more than men but only 43 more for excellent and 130 more for poor.

#### Regional Analysis

**Key Findings:**

1. **Healthcare Utilization by Region:**
   - The western region has the highest average visits (6.37)
   - All regions have roughly the same emergency care usage (0.24-0.28)
   - All regions have the same average hospital visits (0.30) aside from Northwest (0.27)

2. **Demographics by Region:**
   - Income varies by region: West region had the highest income (\\$31,165) and 'Other' had the lowest (\\$21,662)
   - Age distribution: All ages are roughly the same (73-74)
   - Education levels: West has the most years of school (11.48) with 'Other' having the least (9.59)

3. **Regional Distribution:**
   - Population spread:
| Region | Count | Percentage |
|--------|-------|------------|
| Other | 1,614 | 36.6% |
| Midwest | 1,157 | 26.3% |
| Northeast | 837 | 19.0% |
| West | 798 | 18.1% |
   - Most common region: 'Other' (36.6%)

4. **Insurance Coverage by Region:**
| Region | No Insurance | % | Private Insurance | % | Total |
|--------|--------------|---|-------------------|---|-------|
| Midwest | 171 | 14.8% | 986 | 85.2% | 1,157 |
| Northeast | 174 | 20.8% | 663 | 79.2% | 837 |
| Other | 464 | 28.7% | 1,150 | 71.3% | 1,614 |
| West | 176 | 22.1% | 622 | 77.9% | 798 |
| **Total** | **985** | **22.4%** | **3,421** | **77.6%** | **4,406** |

    **Medicaid Coverage by Region**

| Region | Not on Medicaid | % | On Medicaid | % | Total |
|--------|-----------------|---|-------------|---|-------|
| Midwest | 1,114 | 96.3% | 43 | 3.7% | 1,157 |
| Northeast | 768 | 91.8% | 69 | 8.2% | 837 |
| Other | 1,419 | 87.9% | 195 | 12.1% | 1,614 |
| West | 703 | 88.1% | 95 | 11.9% | 798 |
| **Total** | **4,004** | **90.9%** | **402** | **9.1%** | **4,406** |
   - Midwest has the best insurance coverage (85.2%)
   - Western region (11.9%) and 'other' (12.1%) had the highest Medicaid usage.


#### Combined Analysis: Health × Region

**Key Findings:**

1. **Physician Visits:**
   - The West region consistently shows higher physician visits across all health categories
   - People with poor health in the Northeast have the highest visit rate (10.67 visits)
   - Western region with excellent health still visits more (3.37) than other regions with excellent health (3.14-3.44)
   - Pattern: Poor health = 8-11 visits, Average = 5-7 visits, Excellent = 3-4 visits across all regions

2. **Income Patterns:**
   - West region has highest income across ALL health categories
   - Income-health relationship is consistent across regions: excellent health = higher income
   - Income gap between excellent and poor health is largest in West (\\$16,137 difference)
   - "Other" region shows lowest incomes across all health categories
   - Pattern holds everywhere: Better health correlates with higher income (possibly due to ability to work, healthcare access, or lifestyle factors)

3. **Chronic Conditions:**
   - Chronic conditions are remarkably consistent across regions for each health status
   - Poor health: ~2.7 chronic conditions regardless of region
   - Average health: ~1.4 chronic conditions across all regions
   - Excellent health: ~0.75 chronic conditions across all regions
   - No significant regional health disparities in chronic condition burden - suggests health status is the primary driver, not geography

**Overall Insights:**

The interaction between health status and region reveals important patterns:

1. **West Region Advantage**: The West consistently shows better outcomes - higher income, more education, and paradoxically more physician visits (likely preventive care and better healthcare access rather than necessity)

2. **Health-Income Connection**: Across ALL regions, better self-reported health strongly correlates with higher income. This relationship is universal, though the absolute income levels vary by region.

3. **Regional Healthcare Access**: Despite income and education differences across regions, chronic condition patterns by health status remain consistent, suggesting that health outcomes are more influenced by individual health status than geography.

4. **"Other" Region Concerns**: Consistently lowest in income (\\$21,663 average) and education (9.59 years), with highest Medicaid usage (12.1%). This region may represent rural or underserved areas needing targeted healthcare interventions.

5. **Midwest Stability**: Highest private insurance coverage (85.2%), moderate income, and consistent healthcare utilization patterns - represents the most economically stable region in the dataset.

**Key Takeaway**: While regional differences exist in income and education, the fundamental relationship between health status and healthcare utilization remains consistent across geography. This suggests that health interventions should focus on health status categories rather than being region-specific, though economic support may need regional tailoring.

---
# Part 2: Analyze and Cleanse Data

### Tasks 1-3: Setup and Analysis Criteria

Focus on creating distribution tables by various demographic and economic factors.

In [20]:
# Part 2 Tasks 1-2: Import libraries and load data (if starting fresh)

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# If not already loaded, load the data
# df = pd.read_csv('outputs/NSMES1988updated.csv')

# Task 3: Analysis criteria overview
print("="*50)
print("ANALYSIS CRITERIA FOR DISTRIBUTION TABLES:")
print("-"*50)

criteria = {
    'Different types of visits': ['visits', 'nvisits', 'ovisits', 'novisits', 'emergency', 'hospital'],
    'Gender': ['gender'],
    'Marital Status': ['married'],
    'School': ['school'],
    'Income': ['income'],
    'Employment Status': ['employed'],
    'Insurance': ['insurance'],
    'Medical Aid (Medicaid)': ['medicaid']
}

for category, columns in criteria.items():
    print(f"\n{category}:")
    for col in columns:
        if col in df.columns:
            print(f"  ✓ {col}")

---
### Task 4.1: Age and Gender Distribution

Examine how age groups are distributed across genders.

In [23]:
# Distribution Table 1: Age and Gender Distribution
print("="*50)
print("DISTRIBUTION TABLE 1: AGE AND GENDER")
print("-"*50)

# Create age groups for better analysis
df['age_group'] = pd.cut(df['age'], 
                          bins=[65, 70, 75, 80, 85, 110],
                          labels=['66-70', '71-75', '76-80', '81-85', '86+'])

# 1. Count by age group and gender
print("\n1. COUNT BY AGE GROUP AND GENDER:")
print("-"*50)

age_gender_count = pd.crosstab(
    df['age_group'],
    df['gender'],
    margins=True,
    margins_name='Total'
)

print(age_gender_count)

# 2. Percentage distribution
print("\n\n2. PERCENTAGE DISTRIBUTION (by row):")
print("-"*50)

age_gender_pct = pd.crosstab(
    df['age_group'],
    df['gender'],
    normalize='index'
) * 100

print(age_gender_pct.round(1))

# 3. Summary statistics by gender
print("\n\n3. AGE STATISTICS BY GENDER:")
print("-"*50)

age_by_gender = df.groupby('gender')['age'].agg([
    ('Count', 'count'),
    ('Mean', 'mean'),
    ('Median', 'median'),
    ('Std Dev', 'std'),
    ('Min', 'min'),
    ('Max', 'max')
]).round(2)

print(age_by_gender)

# 4. Gender distribution overall
print("\n\n4. OVERALL GENDER DISTRIBUTION:")
print("-"*50)

gender_dist = df['gender'].value_counts()
gender_pct = (df['gender'].value_counts(normalize=True) * 100).round(1)

gender_summary = pd.DataFrame({
    'Count': gender_dist,
    'Percentage': gender_pct.apply(lambda x: f"{x}%")
})

print(gender_summary)

DISTRIBUTION TABLE 1: AGE AND GENDER
--------------------------------------------------

1. COUNT BY AGE GROUP AND GENDER:
--------------------------------------------------
gender     female  male  Total
age_group                     
66-70         897   671   1568
71-75         736   530   1266
76-80         525   321    846
81-85         303   176    479
86+           167    80    247
Total        2628  1778   4406


2. PERCENTAGE DISTRIBUTION (by row):
--------------------------------------------------
gender     female  male
age_group              
66-70        57.2  42.8
71-75        58.1  41.9
76-80        62.1  37.9
81-85        63.3  36.7
86+          67.6  32.4


3. AGE STATISTICS BY GENDER:
--------------------------------------------------
        Count   Mean  Median  Std Dev   Min    Max
gender                                            
female   2628  74.32    73.0     6.46  66.0  109.0
male     1778  73.59    72.0     6.13  66.0  102.0


4. OVERALL GENDER DISTRIBUTION:


---
### Task 4.2: Health Status by Gender

Analyze how self-perceived health varies between genders.

In [25]:
# Distribution Table 2: Health Status by Gender

print("\n" + "="*50)
print("DISTRIBUTION TABLE 2: HEALTH STATUS BY GENDER")
print("-"*50)

# 1. Count by health status and gender
print("\n1. COUNT BY HEALTH STATUS AND GENDER:")
print("-"*50)

health_gender_count = pd.crosstab(
    df['health'],
    df['gender'],
    margins=True,
    margins_name='Total'
)

print(health_gender_count)

# 2. Percentage within each gender (column percentages)
print("\n\n2. HEALTH DISTRIBUTION WITHIN EACH GENDER (%):")
print("-"*50)

health_gender_pct = pd.crosstab(
    df['health'],
    df['gender'],
    normalize='columns'
) * 100

print(health_gender_pct.round(1))

# 3. Healthcare utilization by health and gender
print("\n\n3. AVERAGE PHYSICIAN VISITS BY HEALTH AND GENDER:")
print("-"*50)

health_gender_visits = df.pivot_table(
    values='visits',
    index='health',
    columns='gender',
    aggfunc='mean'
).round(2)

print(health_gender_visits)

# 4. Chronic conditions by health and gender
print("\n\n4. AVERAGE CHRONIC CONDITIONS BY HEALTH AND GENDER:")
print("-"*50)

health_gender_chronic = df.pivot_table(
    values='chronic',
    index='health',
    columns='gender',
    aggfunc='mean'
).round(2)

print(health_gender_chronic)

# 5. Chi-square test insight (basic comparison)
print("\n\n5. GENDER COMPARISON IN HEALTH REPORTING:")
print("-"*50)

for gender in df['gender'].unique():
    gender_data = df[df['gender'] == gender]['health'].value_counts(normalize=True) * 100
    print(f"\n{gender.capitalize()}:")
    for health_status, pct in gender_data.items():
        print(f"  {health_status}: {pct:.1f}%")


DISTRIBUTION TABLE 2: HEALTH STATUS BY GENDER
--------------------------------------------------

1. COUNT BY HEALTH STATUS AND GENDER:
--------------------------------------------------
gender     female  male  Total
health                        
average      2093  1416   3509
excellent     193   150    343
poor          342   212    554
Total        2628  1778   4406


2. HEALTH DISTRIBUTION WITHIN EACH GENDER (%):
--------------------------------------------------
gender     female  male
health                 
average      79.6  79.6
excellent     7.3   8.4
poor         13.0  11.9


3. AVERAGE PHYSICIAN VISITS BY HEALTH AND GENDER:
--------------------------------------------------
gender     female  male
health                 
average      5.75  5.16
excellent    3.23  3.68
poor         9.21  8.39


4. AVERAGE CHRONIC CONDITIONS BY HEALTH AND GENDER:
--------------------------------------------------
gender     female  male
health                 
average      1.44  1.42
excell

---
### Task 4.3: Income Distribution by Gender

Examine income patterns and economic disparities across genders.

In [29]:
# Distribution Table 3: Income Distribution by Gender

print("\n" + "="*50)
print("DISTRIBUTION TABLE 3: INCOME DISTRIBUTION BY GENDER")
print("-"*50)

# Create income groups
df['income_group'] = pd.cut(df['income'],
                             bins=[-20000, 10000, 20000, 30000, 50000, 600000],
                             labels=['<$10k', '$10k-$20k', '$20k-$30k', '$30k-$50k', '$50k+'])

# 1. Count by income group and gender
print("\n1. COUNT BY INCOME GROUP AND GENDER:")
print("-"*50)

income_gender_count = pd.crosstab(
    df['income_group'],
    df['gender'],
    margins=True,
    margins_name='Total'
)

print(income_gender_count)

# 2. Percentage distribution within each gender
print("\n\n2. INCOME DISTRIBUTION WITHIN EACH GENDER (%):")
print("-"*50)

income_gender_pct = pd.crosstab(
    df['income_group'],
    df['gender'],
    normalize='columns'
) * 100

print(income_gender_pct.round(1))

# 3. Income statistics by gender
print("\n\n3. INCOME STATISTICS BY GENDER:")
print("-"*50)

income_by_gender = df.groupby('gender')['income'].agg([
    ('Count', 'count'),
    ('Mean', 'mean'),
    ('Median', 'median'),
    ('Std Dev', 'std'),
    ('Min', 'min'),
    ('Max', 'max')
]).round(2)

# Format income columns as currency
for col in ['Mean', 'Median', 'Std Dev', 'Min', 'Max']:
    income_by_gender[col] = income_by_gender[col].apply(lambda x: f"${x:,.2f}")

print(income_by_gender)

# 4. Additional analysis: Employment and income by gender
print("\n\n4. EMPLOYMENT STATUS AND AVERAGE INCOME BY GENDER:")
print("-"*50)

employment_income = df.groupby(['gender', 'employed'])['income'].mean().round(2)
print(employment_income)

# 5. Insurance coverage and income
print("\n\n5. INSURANCE COVERAGE BY INCOME GROUP AND GENDER:")
print("-"*50)

insurance_income_gender = pd.crosstab(
    [df['income_group'], df['gender']],
    df['insurance']
)

print(insurance_income_gender)

# 6. Employment vs gender
print("\n\n2. EMPLOYMENT DISTRIBUTION WITHIN EACH GENDER (%):")
print("-"*50)

employ_gender_pct = pd.crosstab(
    df['employed'],
    df['gender'],
    normalize='columns'
) * 100

print(employ_gender_pct.round(1))


DISTRIBUTION TABLE 3: INCOME DISTRIBUTION BY GENDER
--------------------------------------------------

1. COUNT BY INCOME GROUP AND GENDER:
--------------------------------------------------
gender        female  male  Total
income_group                     
<$10k            947   303   1250
$10k-$20k        723   559   1282
$20k-$30k        350   340    690
$30k-$50k        360   323    683
$50k+            248   253    501
Total           2628  1778   4406


2. INCOME DISTRIBUTION WITHIN EACH GENDER (%):
--------------------------------------------------
gender        female  male
income_group              
<$10k           36.0  17.0
$10k-$20k       27.5  31.4
$20k-$30k       13.3  19.1
$30k-$50k       13.7  18.2
$50k+            9.4  14.2


3. INCOME STATISTICS BY GENDER:
--------------------------------------------------
        Count        Mean      Median     Std Dev          Min          Max
gender                                                                     
female   

---
### Task 4.4: Regional Income Distribution

Analyze income patterns across different geographic regions.

In [27]:
# Distribution Table 4: Regional Income Distribution

print("\n" + "="*50)
print("DISTRIBUTION TABLE 4: REGIONAL INCOME DISTRIBUTION")
print("-"*50)

# 1. Count by income group and region
print("\n1. COUNT BY INCOME GROUP AND REGION:")
print("-"*50)

income_region_count = pd.crosstab(
    df['income_group'],
    df['region'],
    margins=True,
    margins_name='Total'
)

print(income_region_count)

# 2. Percentage distribution within each region
print("\n\n2. INCOME DISTRIBUTION WITHIN EACH REGION (%):")
print("-"*50)

income_region_pct = pd.crosstab(
    df['income_group'],
    df['region'],
    normalize='columns'
) * 100

print(income_region_pct.round(1))

# 3. Income statistics by region
print("\n\n3. INCOME STATISTICS BY REGION:")
print("-"*50)

income_by_region = df.groupby('region')['income'].agg([
    ('Count', 'count'),
    ('Mean', 'mean'),
    ('Median', 'median'),
    ('Std Dev', 'std'),
    ('Min', 'min'),
    ('Max', 'max')
]).round(2)

# Format as currency
for col in ['Mean', 'Median', 'Std Dev', 'Min', 'Max']:
    income_by_region[col] = income_by_region[col].apply(lambda x: f"${x:,.2f}")

print(income_by_region)

# 4. Regional comparison - which region is wealthiest?
print("\n\n4. REGIONAL INCOME RANKING:")
print("-"*50)

region_ranking = df.groupby('region')['income'].agg(['mean', 'median']).round(2)
region_ranking = region_ranking.sort_values('mean', ascending=False)
region_ranking.columns = ['Average Income', 'Median Income']

for col in region_ranking.columns:
    region_ranking[col] = region_ranking[col].apply(lambda x: f"${x:,.2f}")

print(region_ranking)

# 5. Education level by region (related to income)
print("\n\n5. AVERAGE EDUCATION YEARS BY REGION:")
print("-"*50)

education_by_region = df.groupby('region')['school'].mean().round(2).sort_values(ascending=False)
print(education_by_region)


DISTRIBUTION TABLE 4: REGIONAL INCOME DISTRIBUTION
--------------------------------------------------

1. COUNT BY INCOME GROUP AND REGION:
--------------------------------------------------
region        midwest  northeast  other  west  Total
income_group                                        
<$10k             282        205    567   196   1250
$10k-$20k         362        266    462   192   1282
$20k-$30k         204        129    234   123    690
$30k-$50k         195        139    207   142    683
$50k+             114         98    144   145    501
Total            1157        837   1614   798   4406


2. INCOME DISTRIBUTION WITHIN EACH REGION (%):
--------------------------------------------------
region        midwest  northeast  other  west
income_group                                 
<$10k            24.4       24.5   35.1  24.6
$10k-$20k        31.3       31.8   28.6  24.1
$20k-$30k        17.6       15.4   14.5  15.4
$30k-$50k        16.9       16.6   12.8  17.8
$50k+   

---
### Task 4.5: Age-wise Income Analysis

Examine the relationship between age and income in the elderly population.

In [28]:
# Distribution Table 5: Age-wise Income Analysis

print("\n" + "="*50)
print("DISTRIBUTION TABLE 5: AGE-WISE INCOME ANALYSIS")
print("-"*50)

# 1. Count by age group and income group
print("\n1. COUNT BY AGE GROUP AND INCOME GROUP:")
print("-"*50)

age_income_count = pd.crosstab(
    df['age_group'],
    df['income_group'],
    margins=True,
    margins_name='Total'
)

print(age_income_count)

# 2. Percentage distribution (income within each age group)
print("\n\n2. INCOME DISTRIBUTION WITHIN EACH AGE GROUP (%):")
print("-"*50)

age_income_pct = pd.crosstab(
    df['age_group'],
    df['income_group'],
    normalize='index'
) * 100

print(age_income_pct.round(1))

# 3. Income statistics by age group
print("\n\n3. INCOME STATISTICS BY AGE GROUP:")
print("-"*50)

income_by_age = df.groupby('age_group')['income'].agg([
    ('Count', 'count'),
    ('Mean', 'mean'),
    ('Median', 'median'),
    ('Std Dev', 'std'),
    ('Min', 'min'),
    ('Max', 'max')
]).round(2)

# Format as currency
for col in ['Mean', 'Median', 'Std Dev', 'Min', 'Max']:
    income_by_age[col] = income_by_age[col].apply(lambda x: f"${x:,.2f}")

print(income_by_age)

# 4. Correlation between age and income
print("\n\n4. AGE-INCOME CORRELATION:")
print("-"*50)

correlation = df['age'].corr(df['income'])
print(f"Correlation coefficient: {correlation:.3f}")

if correlation > 0:
    print("→ Positive correlation: Income tends to increase with age")
elif correlation < 0:
    print("→ Negative correlation: Income tends to decrease with age")
else:
    print("→ No correlation: Age and income are independent")

# 5. Income sources by age (employment status)
print("\n\n5. EMPLOYMENT STATUS BY AGE GROUP:")
print("-"*50)

employment_age = pd.crosstab(
    df['age_group'],
    df['employed'],
    normalize='index'
) * 100

print(employment_age.round(1))

# 6. Average income by age and employment
print("\n\n6. AVERAGE INCOME BY AGE GROUP AND EMPLOYMENT STATUS:")
print("-"*50)

income_age_employment = df.pivot_table(
    values='income',
    index='age_group',
    columns='employed',
    aggfunc='mean'
).round(2)

# Format as currency
for col in income_age_employment.columns:
    income_age_employment[col] = income_age_employment[col].apply(lambda x: f"${x:,.2f}")

print(income_age_employment)


DISTRIBUTION TABLE 5: AGE-WISE INCOME ANALYSIS
--------------------------------------------------

1. COUNT BY AGE GROUP AND INCOME GROUP:
--------------------------------------------------
income_group  <$10k  $10k-$20k  $20k-$30k  $30k-$50k  $50k+  Total
age_group                                                         
66-70           345        451        286        285    201   1568
71-75           320        403        204        189    150   1266
76-80           308        236        110        107     85    846
81-85           186        131         64         62     36    479
86+              91         61         26         40     29    247
Total          1250       1282        690        683    501   4406


2. INCOME DISTRIBUTION WITHIN EACH AGE GROUP (%):
--------------------------------------------------
income_group  <$10k  $10k-$20k  $20k-$30k  $30k-$50k  $50k+
age_group                                                  
66-70          22.0       28.8       18.2       18

---
### Task 5: Distribution Tables - Findings Report

#### Table 1: Age and Gender Distribution

**Key Findings:**
- Total population: 4406 individuals
- Gender split: 59.6% female, 40.4% male
- Age range: 66-109 years

**Age Group Distribution:**
- Largest age group: 66-70 with 1568 people
- Gender representation across age groups lean towards female, increasing as the age increases. 14.8% more female for ages 66-70, and 35.2% above 86.
- Mean age: Female 74.32 years, Male 73.59 years

---

#### Table 2: Health Status by Gender

**Key Findings:**
- Overall health distribution: 79.6% average, 12.6% poor, 7.8% excellent
- Male reports better health overall. Average is the same as females but  male reports 8.4% excellent, while female reports 7.3%. 

**Health by Gender:**
- Female: 13.0% poor, 79.6% average, 7.3% excellent
- Male: 11.9% poor, 79.6% average, 8.4% excellent

**Healthcare Usage Pattern:**
- People with poor health average 9.21 physician visits for female and 8.39 physician visits for male.
- People with excellent health average 3.29 visits for female and 3.68 visits for male

---

#### Table 3: Income Distribution by Gender

**Key Findings:**
- Overall mean income: \\$25,935.32
- Female mean income: \\$22,493
- Male mean income: \\$29,377
- Income gap:
  | Income Group | Female | Male | Difference (Female − Male) | % Difference vs Male |
|---------------|:------:|:----:|:--------------------------:|:--------------------:|
| <\\$10k         | 36.0   | 17.0 | 19.0                      | 111.8%              |
| \$10k–\\$20k     | 27.5   | 31.4 | -3.9                      | -12.4%              |
| \$20k–\\$30k     | 13.3   | 19.1 | -5.8                      | -30.4%              |
| \\$30k–\$50k     | 13.7   | 18.2 | -4.5                      | -24.7%              |
| \\$50k+         | 9.4    | 14.2 | -4.8                      | -33.8%              |


**Income Distribution:**
- Most people (27.5% of females and 31.4% of males) earn between \\$-10k\$20k
- 36% of females and 17% of males earn less than \\$10k
- 9.4% of females and 14.2% of men earn more than \\$50k

**Employment and Income:**
- Employed individuals earn \\$14,697 more on average (\$11,328 for females and \\$18,067 for males)
- 7.3% of females employed vs 14.7% of males

---

#### Table 4: Regional Income Distribution

**Key Findings:**
- Wealthiest region: west with average income \\$31,165
- Poorest region: 'other' with average income \\$21,662
- Income disparity: \\$9,503 difference between highest and lowest on average. 

**Regional Patterns:**
- West has the highest education level (11.48 years) and 'other' has the least (9.59 years) 
- 'Other' and midwest have the highest insurance coverage. West has the lowest.

| Region     | Average Income | Median Income |
|-------------|----------------|----------------|
| West        | \\$31,165.05     | \\$20,656.00     |
| Northeast   | \\$26,797.09     | \\$17,413.00     |
| Midwest     | \\$25,136.34     | \\$17,875.00     |
| Other       | \\$21,662.84     | \\$14,220.00     |


---

#### Table 5: Age-wise Income Analysis

**Key Findings:**
- Age-income correlation: -0.073. Negative correlation: Income tends to decrease with age
- Youngest group (66-70): Average income \\$27,488
- Oldest group (86+): Average income \\$23,951

**Employment by Age:**
- Youngest group: 16.8% still employed
- Oldest group: 1.6% still employed
- Most people in all age groups are retired (92.6% not employed)

**Income Sources:**
- Employed individuals: Average \\$36,452
- Not employed (likely retired): Average \\$23,084

---

# Overall Insights


**Cross-Cutting Patterns:**
- Females represent the majority of the older population and show lower income levels despite similar or higher healthcare use.
- Regions with higher education levels (like the West) also have higher average incomes, suggesting a link between education and earnings even in older populations.
- Employment status remains the strongest predictor of higher income, with employed individuals earning roughly 60% more on average.

**Socioeconomic Factors:**
- Lower income is associated with both poorer health and higher physician visits, indicating potential chronic health issues among financially vulnerable groups.
- Females, especially in lower-income or older age brackets, appear more likely to experience economic vulnerability.
- Regional disparities suggest structural differences — the West performs best economically but has lower insurance coverage, while lower-income regions report higher coverage but lower education levels.

**Healthcare Implications:**
- Older, low-income, and female populations may face barriers to preventive care despite frequent physician contact.
- Regional healthcare access and cost differences could be driving the imbalance between income and visit frequency.
- Targeted outreach for low-income seniors, especially in 'Other' and Midwest regions, could improve outcomes and reduce healthcare strain.

**Data Quality Notes:**
- Negative income cases: 3 individuals.
- Some outliers may reflect reporting errors or non-traditional income sources (e.g., losses from investments).
- Overall, data completeness appears high, but follow-up validation for negative or zero-income cases is recommended.